In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
import uvicorn
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from fastapi import FastAPI
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain import PromptTemplate

In [3]:
chat_history=[]
def chatbot(query: str):
    global chat_history
    
    search = GoogleSearchAPIWrapper(k=10)
    tools = Tool(
        name="Google Search",
        description="Search Google for recent results related to Python and React and provide a descriptive answer. Also provide guidance to the user.",
        func=search.run,
    )

    # Create the OpenAI language model
    llm = ChatOpenAI(temperature=0.15, model_name="gpt-3.5-turbo")

    # Create the conversation buffer memory
    memory = ConversationBufferMemory(memory_key="chat_history")

    # Define the chatbot prompt template
    QA_PROMPT_TMPL = '''{bot_name} is a chatbot developed by {company_name} that helps you with Python and React-related questions. It provides users with documentation, installation assistance, code understanding, and other guidance related to Python and React.
    Context:
    {context_str}
    Conversation History:
    {chat_history}
    User:
    {query_str}
    Reply:'''

    prompt = PromptTemplate(
        template=QA_PROMPT_TMPL,
        input_variables=["context_str", "chat_history", "query_str"],
        partial_variables={"bot_name": "codingBOT", "company_name": "XYZ"}
    )

    # Initialize the chatbot agent
    agent_chain = initialize_agent(
        [tools],
        llm,
        agent="conversational-react-description",
        memory=memory,
        verbose=True,
        max_iterations=5,
    )

    # Generate the chatbot response
    chat_history_str = "\n".join(chat_history)
    agent_prompt = prompt.format(
        query_str=query,
        context_str="casual discussion between user and bot about Python or React",
        chat_history=chat_history_str
    )
    
    try:
        response = agent_chain.run(input=agent_prompt)
    except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output: `"):
            response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")

    # Add the user's query and the chatbot's response to the chat history
    chat_history.append(f"User: {query}, Response: {response}")

    if len(chat_history_str) > 3000:
        chat_history.pop(0)

    # Return the chatbot's response
    return response

query = "give me step by step guide to install React in my macOS"
response = chatbot(query)
print(response)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Google Search
Action Input: "step by step guide to install React in macOS"
Observation: Jan 4, 2023 ... Installing React on MacOS - Steps. Before we perform the installation steps, we need to look at the prerequisites needed for installation. 1. Oct 26, 2021 ... How to Install ReactJS on MacOS? ; Download for MacOS ; pop-up ; install confirmation ; Node.js location ; A react project structure. Feb 25, 2020 ... Here, let's go step by step installation of React on mac. Mac: User must have proper access and privilege for installation of software. Apr 1, 2022 ... A step by step guide on how to setup React.js on MacOS. You will install Visual Studio Code, Node.js, NPM, Git, Prettier and ... Step 5. install Xcode command line tools : To install the latest version on Xcode you need to follow the below steps : Open Xcode -> Preferences ... In this article I will walk through the steps needed to install ReactJ